1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림
  
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서로 LLM에 질문과 같이 전달

# 1. 문서의 내용을 읽는다.

In [ ]:
%pip install python-docx

In [ ]:
from docx import Document

document = Document("./tax.docx")
print(f'document == {document}')
full_text = ''
for index, paragraph in enumerate(document.paragraphs):
    print(f'paragraph {index} == {paragraph.text}')
    full_text += f'{paragraph.text}\n'

# 2. 문서를 쪼갠다

In [ ]:
%pip install tiktoken

In [ ]:
import tiktoken

def split_text(full_text, chunk_size) :
  encoder = tiktoken.encoding_for_model("gpt-4o")
  total_encoding = encoder.encode(full_text)
  total_token_count = len(total_encoding)
  text_list = []
  for i in range(0, total_token_count, chunk_size) :
    chunk = total_encoding[i:i + chunk_size]
    decoded = encoder.decode(chunk)
    text_list.append(decoded)
  
  return text_list

In [ ]:
chunk_list = split_text(full_text, 1500)

# 3. 문서 임베딩

In [ ]:
%pip install chromadb

In [ ]:
import chromadb

chroma_client = chromadb.Client()

In [ ]:
import os
from dotenv import load_dotenv
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

load_dotenv()
open_api_key = os.getenv('OPENAI_API_KEY')
openai_embedding = OpenAIEmbeddingFunction(
  api_key=open_api_key)

In [ ]:
collection_name = 'tax_collection2'
tax_collection = chroma_client.get_or_create_collection(collection_name, embedding_function=openai_embedding)

In [ ]:
id_list = []
for index in range(len(chunk_list)) :
  id_list.append(f'tax_doc_chunk_{index}')

In [ ]:
len(id_list) == len(chunk_list)

In [ ]:
tax_collection.add(documents=chunk_list, ids=id_list)

# 4.유사도 검색

In [ ]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_text = tax_collection.query(query_texts=query)

# 5.LLM 질의

In [ ]:
%pip install openai

In [15]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model = "gpt-4o",
  messages=[
    {"role": "system", "content": f"당신은 한국의 소득세 전문가 입니다. 아래 내용을 참고해서 질문에 답변해 주세요. {retrieved_text['documents'][0]}"},
    {"role": "user", "content": query}
  ]
)

In [16]:
response.choices[0].message.content

'연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 몇 가지 단계가 필요합니다. 아래는 일반적인 소득세 계산 절차입니다.\n\n1. **총 급여에서 비과세 소득 등을 제외한 과세 표준 계산**: 소득세는 총 급여 금액이 아닌, 비과세 항목을 제외한 과세 표준에 대해 부과됩니다.\n\n2. **근로소득 공제**: 총 급여액에서 일정 금액을 공제합니다. 2023년 기준으로, 5천만원의 총 급여에 대한 근로소득 공제는 다음과 같이 계산할 수 있습니다.\n   - 5,000만원의 경우: 1200만원 + (총 급여액 - 4,500만원) × 15% = 1200만원 + (5,000만원 - 4,500만원) × 0.15 = 1200만원 + 75만원 = 1,275만원\n\n3. **과세 표준 계산**: \n   - 과세 표준 = 총 급여 - 근로소득 공제\n   - 과세 표준 = 5,000만원 - 1,275만원 = 3,725만원\n\n4. **세율 적용**: 과세 표준에 해당하는 세율을 적용합니다. 한국의 소득세율은 구간별로 다르게 적용됩니다. 2023년 기준 대략적인 세율은 다음과 같습니다:\n   - 1,200만원 이하: 6%\n   - 1,200만원 초과 ~ 4,600만원 이하: 15%\n   - 4,600만원 초과 ~ 8,800만원 이하: 24%\n\n5. **소득세 계산**:\n   - 1,200만원 초과액에 대한 과세: (3,725만원 - 1,200만원) × 15% = 2,525만원 × 0.15 = 378.75만원\n   - 1,200만원 구간에 대한 세금: 1,200만원 × 6% = 72만원\n   - 총 소득세 = 378.75만원 + 72만원 = 450.75만원\n\n6. **기타 세액 공제**: 계산된 소득세에서 각종 세액 공제 항목(인적 공제, 보험료 공제 등)이 적용되면 실제 납부할 소득세가 결정됩니다.\n\n따라서, 구체적인 공제 항목과 액수를 고려하지 않은 상태에서 기본적으로 예상되는 소득세는 약 450.75만원입니다. 최종 소득세는 개인의 공제 사항 